In [ ]:
import os,sys
import cv2
from tqdm import tqdm
import re
import random


import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


from tensorflow.keras import models, Sequential, layers, regularizers, Model, Input
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import image_dataset_from_directory, get_file, plot_model
import tensorflow.keras.applications.resnet50 as resnet50
import tensorflow.keras.applications.inception_v3 as inception_v3

from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
from tensorflow.keras.models import load_model as keras_load_model

from sklearn.metrics import classification_report, confusion_matrix


# Dataset - Get y_true and y_pred

In [ ]:
path_train = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/train'
path_test = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/test'

In [ ]:
# # TRAIN DATASET
# train_ds = image_dataset_from_directory(directory=path_train,
#                                       labels='inferred',
#                                       label_mode="categorical",
#                                       validation_split=0.2,
#                                       subset="training",
#                                       seed=123,
#                                       image_size=(224, 224),
#                                       batch_size=32)   

# # Preprocess X in the train_dataset
# prep_train_ds = train_ds.map(lambda x, y: (inception_preprocess_input(x), y))
# # train_ds_prepro = train_dataset.map(preprocess)

# ###########
# #VALIDATION DATASET
# validation_ds = image_dataset_from_directory(directory=path_train,
#                                       labels='inferred',
#                                       label_mode="categorical",
#                                       validation_split=0.2,
#                                       subset="validation",
#                                       seed=123,
#                                       image_size=(224, 224),
#                                       batch_size=32)
                                         
# # Preprocess X in the val_dataset
# prep_val_ds = validation_ds.map(lambda x, y: (inception_preprocess_input(x), y))

############
# TEST DATASET
test_ds = image_dataset_from_directory(directory=path_test,
                                            labels='inferred',
                                            label_mode="categorical",
                                            validation_split=None,
                                            subset=None,
                                            shuffle=False,            #set shuffle=False 
                                            seed=123,
                                            image_size=(224, 224),
                                            batch_size=32) 

# Preprocess X in the test_dataset
prep_test_ds = test_ds.map(lambda x, y: (inception_preprocess_input(x), y))

In [ ]:
# Load Inception_V3 model
model_path="/kaggle/input/inception-model/inception_model.h5"    #use load_selected_model function in registry.py
model = keras_load_model(model_path, compile=False)
model.summary()

In [ ]:
# Compile
opt = optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Plot the diagram
plot_model(model, show_shapes=True)

In [ ]:
# Evaluate the combined model on the test dataset
res = model.evaluate(prep_test_ds) 
test_accuracy = res[-1]
print(f"test_accuracy_model_1 = {round(test_accuracy,2)*100} %")

In [ ]:
# Get the predictions on the test images
predictions = model.predict(prep_test_ds, batch_size=32)   #steps=269: Total number of steps (batches of samples) before declaring the prediction round finished.
predictions, predictions.shape

In [ ]:
# Get y_pred 
y_pred = np.argmax(predictions, axis=1)  #returns the indices of the maximum values along an axis
y_pred, y_pred.shape

In [ ]:
# Unbatch the test dataset

unbatched_test_ds = prep_test_ds.unbatch()
# images = list(unbatched_test_ds.map(lambda x, y: x))   #images
labels = list(unbatched_test_ds.map(lambda x, y: y))   #y_true

In [ ]:
# Get y_true

labels = np.array(labels)
print(labels)

y_true = np.argmax(labels, axis=1)    #argmax return the indices of the maximum values along an axis.
y_true, y_true.shape, y_pred.shape

In [ ]:
# Get the class names
class_names = test_ds.class_names
class_names = [re.findall('n\d{8}-(.*)', i)[0].capitalize() for i in class_names]
class_names

# Classification report

In [ ]:
# Classification report for all 120 classes
report_120_classes = classification_report(y_true=y_true, y_pred=y_pred, target_names=class_names)
print(report_120_classes)

#precision = TP/(TP + FP) = accuracy of positive predictions (what percent of our predictions were correct?)
#recall = TP/(TP+FN) = fraction of positives that were correctly identified (what percent of the positive cases did we catch?)
#f1-score = 2*(Recall * Precision) / (Recall + Precision) = what percent of positive predictions were correct?
#support = the number of actual occurrences of the class in the test dataset.

In [ ]:
# Change style
report = classification_report(y_true=y_true, y_pred=y_pred, target_names=class_names, output_dict=True) 
report = pd.DataFrame(report).transpose()[:120].reset_index().rename(columns = {'index':'breeds'})

properties = {"color": "#241149"}
cell_hover = {"selector": "td:hover","props": [("background-color", "#f7b8b3")]}
headers = {"selector": "th:not(.index_name)","props": "background-color: #241149; color: white;"}
report.style.format(precision=2).set_properties(**properties).set_table_styles([cell_hover, headers]).hide_index()


# Confusion matrix

In [ ]:
# Confusion Matrix for all 120 classes 
print('Confusion Matrix')
cm = confusion_matrix(y_true=y_true, y_pred=y_pred)     
df = pd.DataFrame(cm, columns=class_names, index=class_names)  
df

In [ ]:
# Confusion Matrix for all 120 classes 
fig = plt.figure(figsize=(80,80))
sns.heatmap(df, annot=True)


In [ ]:
# Save the figure
fig_cm = fig.get_figure()
fig_cm.savefig("confusion_matrix.png")

# Horizontal Bar Charts 

In [ ]:
import plotly.express as px

In [ ]:
report[["precision","recall","f1-score","support"]] = report[["precision","recall","f1-score","support"]].round(decimals=3)
report

### Horizontal Bar Chart for The Least Regconized Breeds 

In [ ]:
# Add a horizontal bar chart for the least recognized breeds

least_recognized_breeds = report.sort_values(by="recall")[report.recall < 0.6]
least_recognized_breeds = least_recognized_breeds.rename(columns = {'index':'breeds'})

fig1= px.bar(least_recognized_breeds, x="recall", y="breeds", orientation="h",
                hover_data=['precision', 'f1-score'], height=400, width=650, color="recall",
                color_continuous_scale="pubu", range_color=[0.5,1], template='simple_white')
fig1.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)', 'paper_bgcolor': 'rgba(0,0,0,0)'})
fig1.show()

### Horizontal Bar Chart for The Top Regconized Breeds

In [ ]:
# Add a horizontal bar chart for the top recognized breeds

top_recognized_breeds = report.sort_values(by="recall")[report.recall > 0.98]
top_recognized_breeds = top_recognized_breeds.rename(columns = {'index':'breeds'})

fig2 = px.bar(top_recognized_breeds, x="recall", y="breeds", orientation="h",
                hover_data=['precision', 'f1-score'], height=400, width=650, color="recall",
                color_continuous_scale="pubu", range_color=[0.5,1], template='simple_white')
fig2.update_layout({'plot_bgcolor': 'rgba(0,0,0,0)', 'paper_bgcolor': 'rgba(0,0,0,0)'})
fig2.show()